In [1]:
import numpy as np

from sympy import symbols
from sympy.logic.boolalg import to_cnf, Not, And, to_int_repr

In [2]:
A, B, C, D = symbols('A B C D')

In [3]:
formula1 = to_cnf(~(A | B) | D)
formula1

(D | ~A) & (D | ~B)

In [4]:
formula2 = to_cnf(~(A | C) | B | ~D)
formula2

(B | ~A | ~D) & (B | ~C | ~D)

In [5]:
formula3 = to_cnf(~(A | ~C) | B | ~ (D | ~A))
formula3

(A | B | C) & (A | B | ~A) & (B | C | ~D) & (B | ~A | ~D)

In [6]:
formula3.free_symbols

{A, B, C, D}

In [7]:
formula3.args

(A | B | C, A | B | ~A, B | C | ~D, B | ~A | ~D)

In [8]:
[list(fo.args) for fo in formula3.args]

[[A, B, C], [A, B, ~A], [B, C, ~D], [B, ~A, ~D]]

In [9]:
to_int_repr(formula3.args, [A, B, C, D])

[{1, 2, 3}, {-1, 1, 2}, {-4, 2, 3}, {-4, -1, 2}]

In [10]:
kb = And(formula2, formula3)
kb 

(A | B | C) & (A | B | ~A) & (B | C | ~D) & (B | ~A | ~D) & (B | ~C | ~D)

In [11]:
type(kb)

And

In [12]:
to_cnf(kb, simplify=True)

(B | ~D) & (A | B | C)

In [13]:
not_phi = to_cnf(Not(formula1))
not_phi

~D & (A | B) & (A | ~D) & (B | ~D)

In [14]:
to_cnf(And(kb, not_phi), simplify=True)

~D & (A | B)

In [15]:
to_cnf(And(kb, not_phi), simplify=False)

~D & (A | B) & (A | ~D) & (B | ~D) & (A | B | C) & (A | B | ~A) & (B | C | ~D) & (B | ~A | ~D) & (B | ~C | ~D)

In [16]:
to_int_repr(And(kb, not_phi).args, [A, B, C, D])

[{-4},
 {1, 2},
 {-4, 1},
 {-4, 2},
 {1, 2, 3},
 {-1, 1, 2},
 {-4, 2, 3},
 {-4, -1, 2},
 {-4, -3, 2}]

In [51]:
from heapq import heappush, heapify
from collections import defaultdict

def resolution(kb, phi, simplify=False):
    kb = to_cnf(kb)
    not_phi = to_cnf(Not(phi))
    
    clauses = to_cnf(And(kb, not_phi), simplify=simplify) # already sorted by number of literals
    #clauses = [list(clause.args) for clause in clauses.args] # list of clauses, each composed by list of literals
    clauses = to_int_repr(clauses.args, clauses.free_symbols) # already sorted by number of literals
    #clauses = heapify(clauses)
    print('clauses:', clauses)    
    
    # print(np.array(clauses).flatten())
    # literals = set(clauses)
    # print('literals', literals)
    
    literals_clauses = defaultdict(list)
    for clause in clauses:
        # TODO: delete clauses containing both A and notA
        for literal in clause:
            heappush(literals_clauses[literal], (len(clause), clause))
            #literals_clauses[literal].append((len(clause), clause_id))
            
    #print(literals_clauses)        
     
    # not necessary: already sorted    
    # for literal, clauses in literals_clauses.items():
    #    clauses.sort(key=clauses[0])
    
    print('literals:', literals_clauses.keys())
    print(literals_clauses)
    
    i = 0
    while(i < len(clauses)):
        print('clauses:', clauses)  
        for literal in clauses[i]:
            print('current clause:', clauses[i])
            if literals_clauses[- literal]:
                print('literal:', literal, '--', 'possible clauses:', [claus[-1] for claus in literals_clauses[- literal]])
                other_clause = literals_clauses[- literal][0][-1]
                new_clause = resolve(clauses[i], other_clause)
                if is_empty(new_clause):
                    return True
                if new_clause in clauses:
                    continue
                clauses.insert(0, new_clause) # TODO: sort by number of literals
                heappush(literals_clauses[literal], (len(clause), clause)) # TODO: sort also this accordingly
                i = -1 
                break
        i = i + 1
    print('final clauses:', clauses)    
    return False    

def resolve(clause1, clause2):
    print('resolving:', clause1, clause2)
    for literal1 in clause1:
        for literal2 in clause2:
            if literal1 + literal2 == 0:
                new_clause1 = clause1 - {literal1}
                new_clause2 = clause2 - {literal2}
                return new_clause1.union(new_clause2)
    return None

def is_empty(clause):
    return not clause

In [52]:
resolution(kb, phi=formula1, simplify=True)

clauses: [{-3}, {1, 2}]
literals: dict_keys([-3, 1, 2])
defaultdict(<class 'list'>, {-3: [(1, {-3})], 1: [(2, {1, 2})], 2: [(2, {1, 2})]})
clauses: [{-3}, {1, 2}]
current clause: {-3}
clauses: [{-3}, {1, 2}]
current clause: {1, 2}
current clause: {1, 2}
final clauses: [{-3}, {1, 2}]


False

In [53]:
resolution(kb, phi=formula1, simplify=False)

clauses: [{-4}, {1, 3}, {1, -4}, {3, -4}, {1, 2, 3}, {1, 3, -1}, {2, 3, -4}, {3, -4, -1}, {3, -4, -2}]
literals: dict_keys([-4, 1, 3, 2, -1, -2])
defaultdict(<class 'list'>, {-4: [(1, {-4}), (2, {1, -4}), (2, {3, -4}), (3, {2, 3, -4}), (3, {3, -4, -1}), (3, {3, -4, -2})], 1: [(2, {1, 3}), (2, {1, -4}), (3, {1, 2, 3}), (3, {1, 3, -1})], 3: [(2, {1, 3}), (2, {3, -4}), (3, {1, 2, 3}), (3, {1, 3, -1}), (3, {2, 3, -4}), (3, {3, -4, -1}), (3, {3, -4, -2})], 2: [(3, {1, 2, 3}), (3, {2, 3, -4})], -1: [(3, {1, 3, -1}), (3, {3, -4, -1})], -2: [(3, {3, -4, -2})]})
clauses: [{-4}, {1, 3}, {1, -4}, {3, -4}, {1, 2, 3}, {1, 3, -1}, {2, 3, -4}, {3, -4, -1}, {3, -4, -2}]
current clause: {-4}
clauses: [{-4}, {1, 3}, {1, -4}, {3, -4}, {1, 2, 3}, {1, 3, -1}, {2, 3, -4}, {3, -4, -1}, {3, -4, -2}]
current clause: {1, 3}
literal: 1 -- possible clauses: [{1, 3, -1}, {3, -4, -1}]
resolving: {1, 3} {1, 3, -1}
current clause: {1, 3}
clauses: [{-4}, {1, 3}, {1, -4}, {3, -4}, {1, 2, 3}, {1, 3, -1}, {2, 3, -4}, {3,

False

In [54]:
r, p, s, = symbols('R P S')
resolution(kb = (~r | p | s) & (~ p | r) & (~ s | r) & ~r, phi = ~p)

clauses: [{3}, {-1}, {1, -3}, {1, -2}, {2, 3, -1}]
literals: dict_keys([3, -1, 1, -3, -2, 2])
defaultdict(<class 'list'>, {3: [(1, {3}), (3, {2, 3, -1})], -1: [(1, {-1}), (3, {2, 3, -1})], 1: [(2, {1, -3}), (2, {1, -2})], -3: [(2, {1, -3})], -2: [(2, {1, -2})], 2: [(3, {2, 3, -1})]})
clauses: [{3}, {-1}, {1, -3}, {1, -2}, {2, 3, -1}]
current clause: {3}
literal: 3 -- possible clauses: [{1, -3}]
resolving: {3} {1, -3}
clauses: [{1}, {3}, {-1}, {1, -3}, {1, -2}, {2, 3, -1}]
current clause: {1}
literal: 1 -- possible clauses: [{-1}, {2, 3, -1}]
resolving: {1} {-1}


True